In [127]:
import numpy as np
import pandas as pd
import os
import requests
import bs4
import json
import seaborn as sns
import re
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator
pd.set_option('mode.chained_assignment', None)

In [128]:
%store -r matches
%store -r mids
%store -r tops
%store -r adcs
%store -r sups
%store -r jgs
%store -r lol
%store -r only_exp

In [129]:
dic = {}
def helper_score(row):
    player = row['player']
    pos = row['position']
    if player in dic:
        score = dic[player]
    else:
        if pos == 'top':
            if player in tops.index:
                score = tops.loc[player]['player_score']
            else:
                return
        if pos == 'jng':
            if player in jgs.index:
                score = jgs.loc[player]['player_score']
            else:
                return
        if pos == 'mid':
            if player in mids.index:
                score = mids.loc[player]['player_score']
            else:
                return
        if pos == 'bot':
            if player in adcs.index:
                score = adcs.loc[player]['player_score']
            else:
                return
        if pos == 'sup':
            if player in sups.index:
                score = sups.loc[player]['player_score']
            else:
                return
        dic[player] = score
    return score
only_exp['player_score'] = only_exp.apply(helper_score, axis=1)

In [130]:
only_exp.columns.to_list()

['gameid',
 'datacompleteness',
 'url',
 'league',
 'year',
 'split',
 'playoffs',
 'date',
 'game',
 'patch',
 'playerid',
 'side',
 'position',
 'player',
 'team',
 'champion',
 'ban1',
 'ban2',
 'ban3',
 'ban4',
 'ban5',
 'gamelength',
 'result',
 'kills',
 'deaths',
 'assists',
 'teamkills',
 'teamdeaths',
 'doublekills',
 'triplekills',
 'quadrakills',
 'pentakills',
 'firstblood',
 'firstbloodkill',
 'firstbloodassist',
 'firstbloodvictim',
 'team kpm',
 'ckpm',
 'firstdragon',
 'dragons',
 'opp_dragons',
 'elementaldrakes',
 'opp_elementaldrakes',
 'infernals',
 'mountains',
 'clouds',
 'oceans',
 'dragons (type unknown)',
 'elders',
 'opp_elders',
 'firstherald',
 'heralds',
 'opp_heralds',
 'firstbaron',
 'barons',
 'opp_barons',
 'firsttower',
 'towers',
 'opp_towers',
 'firstmidtower',
 'firsttothreetowers',
 'inhibitors',
 'opp_inhibitors',
 'damagetochampions',
 'dpm',
 'damageshare',
 'damagetakenperminute',
 'damagemitigatedperminute',
 'wardsplaced',
 'wpm',
 'wardskill

In [190]:
X = only_exp[['league', 'playoffs', 'side', 'player', 'team', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'player_score']]
y = only_exp.result
X['ban1'] = X['ban1'].fillna('None')
X['ban2'] = X['ban2'].fillna('None')
X['ban3'] = X['ban3'].fillna('None')
X['ban4'] = X['ban4'].fillna('None')
X['ban5'] = X['ban5'].fillna('None')
X['player_score'] = X['player_score'].fillna(0)

In [238]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

preproc = ColumnTransformer(transformers=[('ohe', OneHotEncoder(handle_unknown='ignore'), ['league', 'side', 'player', 'team', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5'])])

pl = Pipeline(steps=[('preprocessing', preproc), ('classifier', DecisionTreeClassifier(max_depth=60))])
pl.fit(X_train, y_train)
pl.score(X_test, y_test)

0.8711425206124853

In [239]:
X

,league,playoffs,side,player,team,champion,ban1,ban2,ban3,ban4,ban5,player_score
120,LPL,0,Blue,TheShy,Invictus Gaming,Aatrox,Pantheon,Nautilus,Elise,Gangplank,Mordekaiser,53.086264
121,LPL,0,Blue,Ning,Invictus Gaming,Qiyana,Pantheon,Nautilus,Elise,Gangplank,Mordekaiser,-5.692472
122,LPL,0,Blue,Rookie,Invictus Gaming,Rumble,Pantheon,Nautilus,Elise,Gangplank,Mordekaiser,6.555608
123,LPL,0,Blue,Puff,Invictus Gaming,Senna,Pantheon,Nautilus,Elise,Gangplank,Mordekaiser,-6.587265
124,LPL,0,Blue,Southwind,Invictus Gaming,Braum,Pantheon,Nautilus,Elise,Gangplank,Mordekaiser,32.768567
...,...,...,...,...,...,...,...,...,...,...,...,...
101705,LCS,1,Red,Solo,FlyQuest,Gangplank,Nidalee,Caitlyn,Zilean,Bard,Tahm Kench,44.324078
101706,LCS,1,Red,Santorin,FlyQuest,Sett,Nidalee,Caitlyn,Zilean,Bard,Tahm Kench,39.785957
101707,LCS,1,Red,PowerOfEvil,FlyQuest,Orianna,Nidalee,Caitlyn,Zilean,Bard,Tahm Kench,65.188237
101708,LCS,1,Red,WildTurtle,FlyQuest,Ashe,Nidalee,Caitlyn,Zilean,Bard,Tahm Kench,1.248266
